In [1]:
import cobra_postgres
import cobra_datamodelfabric
import uuid
import time
import cobra_jobregistry
import cobra_gdal

In [2]:
reg = cobra_jobregistry.cobra_jobregistry()

In [ ]:
files = ['gis_osm_buildings_a_free_1.shp',
'gis_osm_landuse_a_free_1.shp',
'gis_osm_natural_a_free_1.shp',
'gis_osm_natural_free_1.shp',
'gis_osm_places_a_free_1.shp',
'gis_osm_places_free_1.shp',
'gis_osm_pofw_a_free_1.shp',
'gis_osm_pofw_free_1.shp',
'gis_osm_pois_a_free_1.shp',
'gis_osm_pois_free_1.shp',
'gis_osm_railways_free_1.shp',
'gis_osm_roads_free_1.shp',
'gis_osm_traffic_a_free_1.shp',
'gis_osm_traffic_free_1.shp',
'gis_osm_transport_a_free_1.shp',
'gis_osm_transport_free_1.shp',
'gis_osm_water_a_free_1.shp',
'gis_osm_waterways_free_1.shp']

In [3]:
reg.create_shape2pg_job(f'/download/gis_osm_water_a_free_1.shp', job_name='Water2')

In [ ]:
for a_file in files:
    reg.create_shape2pg_job(f'/download/{a_file}')

In [4]:
gdal = cobra_gdal.cobra_gdal(schema='osm', run_in_loop=True)

Start main loop


Warning 1: Geometry to be inserted is of type Multi Polygon, whereas the layer geometry type is Polygon.
Insertion is likely to fail
ERROR 1: COPY statement failed.
ERROR:  Geometry type (MultiPolygon) does not match column type (Polygon)
CONTEXT:  COPY gis_osm_water_a_free_1, line 852, column wkb_geometry: "0106000020E6100000110000000103000000010000004A000000A922CB38FC341BC0C7FBCCB458EE4E4010E03EBCF8341BC0..."



Start main loop


UnboundLocalError: local variable 'job_type' referenced before assignment

In [ ]:
gdal.execute_shape2pg('/download/gis_osm_water_a_free_1.shp', '2c78cc5a-82b4-42a1-a6af-68094bccd807')